In [1]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, FactorRange, Whisker
from bokeh.models.tools import HoverTool
import itertools 
import bokeh.palettes
from bokeh.transform import factor_cmap
import math 
import pandas as pd
import bokeh_catplot

df = pd.read_csv('Results/bench_time_final_35.csv')
df = df.replace(-1, 0)
print(df)

       runtime image_name ressource_limit  repeat  real_time  user_time  \
0      default     alpine         default       1      0.754      0.059   
1      default     alpine         default       1      0.549      0.055   
2      default     alpine         default      10      0.785      0.059   
3      default     alpine         default      10      0.519      0.048   
4      default     alpine         default      11      0.738      0.051   
...        ...        ...             ...     ...        ...        ...   
11755    runsc     ubuntu             min       7      0.435      0.044   
11756    runsc     ubuntu             min       8      0.845      0.043   
11757    runsc     ubuntu             min       8      0.415      0.040   
11758    runsc     ubuntu             min       9      0.845      0.056   
11759    runsc     ubuntu             min       9      0.535      0.052   

       sys_time                                           filepath  
0         0.000  /opt/auswertu

In [2]:
boot_df = df[df['filepath'].str.contains("create_time")]
print(boot_df)

remove_df = df[df['filepath'].str.contains("rm_time")]
print(remove_df)

       runtime image_name ressource_limit  repeat  real_time  user_time  \
0      default     alpine         default       1      0.754      0.059   
2      default     alpine         default      10      0.785      0.059   
4      default     alpine         default      11      0.738      0.051   
6      default     alpine         default      12      0.743      0.046   
8      default     alpine         default      13      0.831      0.055   
...        ...        ...             ...     ...        ...        ...   
11750    runsc     ubuntu             min       5      0.771      0.059   
11752    runsc     ubuntu             min       6      0.859      0.053   
11754    runsc     ubuntu             min       7      0.882      0.058   
11756    runsc     ubuntu             min       8      0.845      0.043   
11758    runsc     ubuntu             min       9      0.845      0.056   

       sys_time                                           filepath  
0         0.000  /opt/auswertu

In [12]:
grouped_std = boot_df.groupby(['image_name', 'runtime', 'ressource_limit'])['real_time'].std().reset_index(name='std')
grouped_mean = boot_df.groupby(['image_name', 'runtime', 'ressource_limit'])['real_time'].mean().reset_index(name='mean')
#print(grouped_std)
#print(grouped_mean)

boot_grouped = pd.merge(grouped_mean, grouped_std, on=['image_name', 'runtime', 'ressource_limit'])
#print(grouped)

# Quantiel für 35 - 1 = 34 -> 2.7284
# Quelle https://www.staff.uni-mainz.de/klenke/vorlesungen/vorl_ws19-2/tabellenwerk.pdf S7

boot_grouped['konf_intervall_unten'] = boot_grouped['mean'] - boot_grouped['std']  / math.sqrt(35) * 1.6909
boot_grouped['konf_intervall_oben'] = boot_grouped['mean'] + boot_grouped['std']  / math.sqrt(35) *  1.6909
boot_grouped['abweichung']=boot_grouped['std']  / math.sqrt(35) * 1.6909 * 100 / boot_grouped['mean'] 
boot_grouped['type'] = 'up'

print(boot_grouped)

    image_name  runtime ressource_limit      mean       std  \
0       alpine  default         default  0.788686  0.037588   
1       alpine  default             max  0.811857  0.037778   
2       alpine  default             min  0.799886  0.035303   
3       alpine     kata         default  1.389314  0.037746   
4       alpine     kata             max  1.810829  0.064149   
..         ...      ...             ...       ...       ...   
163     ubuntu   katafc             max  2.659543  0.149942   
164     ubuntu   katafc             min  2.584886  0.048560   
165     ubuntu    runsc         default  0.842657  0.040987   
166     ubuntu    runsc             max  0.845229  0.041060   
167     ubuntu    runsc             min  0.840629  0.037569   

     konf_intervall_unten  konf_intervall_oben  abweichung type  
0                0.777942             0.799429    1.362169   up  
1                0.801060             0.822655    1.329978   up  
2                0.789796             0.80997

In [9]:
grouped_std = remove_df.groupby(['image_name', 'runtime', 'ressource_limit'])['real_time'].std().reset_index(name='std')
grouped_mean = remove_df.groupby(['image_name', 'runtime', 'ressource_limit'])['real_time'].mean().reset_index(name='mean')
#print(grouped_std)
#print(grouped_mean)

remove_grouped = pd.merge(grouped_mean, grouped_std, on=['image_name', 'runtime', 'ressource_limit'])
#print(grouped)

# Quantiel für 35 - 1 = 34 -> 2.7284
# Quelle https://www.staff.uni-mainz.de/klenke/vorlesungen/vorl_ws19-2/tabellenwerk.pdf S7

remove_grouped['konf_intervall_unten'] = remove_grouped['mean'] - remove_grouped['std']  / math.sqrt(35) * 1.6909
remove_grouped['konf_intervall_oben'] = remove_grouped['mean'] + remove_grouped['std']  / math.sqrt(35) *  1.6909
remove_grouped['abweichung']=remove_grouped['std']  / math.sqrt(35) * 1.6909 * 100 / remove_grouped['mean'] 
remove_grouped['type'] = 'rm'

print(remove_grouped.to_string())

         image_name  runtime ressource_limit      mean       std  konf_intervall_unten  konf_intervall_oben  abweichung type
0            alpine  default         default  0.493714  0.042216              0.481648             0.505780    2.443883   rm
1            alpine  default             max  0.493057  0.044798              0.480253             0.505861    2.596814   rm
2            alpine  default             min  0.451543  0.043925              0.438988             0.464097    2.780351   rm
3            alpine     kata         default  0.691914  0.084008              0.667904             0.715925    3.470175   rm
4            alpine     kata             max  0.727086  0.074514              0.705788             0.748383    2.929123   rm
5            alpine     kata             min  0.639486  0.044386              0.626800             0.652172    1.983787   rm
6            alpine   katafc         default  0.635257  0.171241              0.586314             0.684200    7.704475   rm


In [19]:


grouped_std_mean = pd.concat([remove_grouped, boot_grouped], sort=False)
grouped_std_mean = grouped_std_mean.reset_index()
grouped_std_mean = grouped_std_mean.drop(['index'], axis=1)

grouped_std_mean['docker_abweichung'] = 0

for index, row in grouped_std_mean.iterrows():
    docker_mean = grouped_std_mean.loc[(grouped_std_mean['runtime'] == 'default') & (grouped_std_mean['type'] == row['type']) & (grouped_std_mean['image_name'] == row['image_name']) & (grouped_std_mean['ressource_limit'] == row['ressource_limit']), ['mean']].reset_index().get_value(0, 'mean')
    #print(index)
    #print(row['mean'] * 100 / docker_mean)
    grouped_std_mean.loc[index, 'docker_abweichung'] = row['mean'] * 100 / docker_mean

grouped_std_mean = grouped_std_mean.round(2)

print(grouped_std_mean.to_string())

export_csv = grouped_std_mean.to_csv (r'Results/time_final_35_Gewichtung_Docker.csv', index = None, header=True, decimal=',')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


         image_name  runtime ressource_limit  mean   std  konf_intervall_unten  konf_intervall_oben  abweichung type  docker_abweichung
0            alpine  default         default  0.49  0.04                  0.48                 0.51        2.44   rm             100.00
1            alpine  default             max  0.49  0.04                  0.48                 0.51        2.60   rm             100.00
2            alpine  default             min  0.45  0.04                  0.44                 0.46        2.78   rm             100.00
3            alpine     kata         default  0.69  0.08                  0.67                 0.72        3.47   rm             140.14
4            alpine     kata             max  0.73  0.07                  0.71                 0.75        2.93   rm             147.46
5            alpine     kata             min  0.64  0.04                  0.63                 0.65        1.98   rm             141.62
6            alpine   katafc         default  0.

\begin{tabular}{llllrrrr}
\toprule
{} &  runtime &      image\_name & ressource\_limit &      mean &  standartabweichung &  konf\_intervall\_unten &  konf\_intervall\_oben \\
\midrule
0   &  default &          alpine &         default &  0.788686 &            0.037588 &              0.771351 &             0.806021 \\
1   &  default &          alpine &             max &  0.811857 &            0.037778 &              0.794434 &             0.829280 \\
2   &  default &          alpine &             min &  0.799886 &            0.035303 &              0.783605 &             0.816167 \\
3   &     kata &          alpine &         default &  1.389314 &            0.037746 &              1.371906 &             1.406722 \\
4   &     kata &          alpine &             max &  1.810829 &            0.064149 &              1.781244 &             1.840413 \\
5   &     kata &          alpine &             min &  1.441886 &            0.046090 &              1.420630 &             1.463142 \\
6   & 

In [6]:
########
# Sum Boottime
#######

group_sum=grouped.groupby(['ressource_limit', 'runtime'])['mean'].sum().reset_index(name='sum_mean')

#latex print
printtbl = group_sum[['ressource_limit', 'runtime', 'sum_mean']]
print(printtbl.to_latex())


source = ColumnDataSource(pd.DataFrame(group_sum))
#print(source.data)

runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
#print(runtimes)

ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)

x = [ (ressource_limit, runtime) for ressource_limit in ressource_limits for runtime in runtimes ]
print(x)

source = ColumnDataSource(data=dict(x=x, mean=source.data['sum_mean'].tolist()))
print(source.data)
p = figure(x_range=FactorRange(*x), width=1000)

# use the palette to colormap based on the the x[1:2] values
fill_color=factor_cmap('x', palette=bokeh.palettes.d3['Category10'][5], factors=runtime, start=1, end=2)

p.vbar(x='x', top='mean', source=source, width=0.70, color=fill_color)



p.title.text ='Boottime in Summe'
p.xaxis.axis_label = 'Ressourcenlimit & Runtime'
p.yaxis.axis_label = 'Boottime in Sekunden'
p.xaxis.axis_label_text_font_size = '11pt'
p.yaxis.axis_label_text_font_size = '11pt'

output_notebook()

show(p)

\begin{tabular}{lllr}
\toprule
{} & ressource\_limit &  runtime &   sum\_mean \\
\midrule
0  &         default &  default &  10.567229 \\
1  &         default &     kata &  17.197743 \\
2  &         default &   katafc &  33.702629 \\
3  &         default &    runnc &   7.639657 \\
4  &         default &    runsc &  11.089657 \\
5  &             max &  default &  10.596629 \\
6  &             max &     kata &  22.802943 \\
7  &             max &   katafc &  33.449200 \\
8  &             max &    runnc &   7.642171 \\
9  &             max &    runsc &  11.157343 \\
10 &             min &  default &  10.517971 \\
11 &             min &     kata &  17.981857 \\
12 &             min &   katafc &  33.023829 \\
13 &             min &    runnc &   7.619486 \\
14 &             min &    runsc &  11.111400 \\
\bottomrule
\end{tabular}

[('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'kata

NameError: name 'runtime' is not defined

In [7]:
######
# Boottime indivduell
####

source = ColumnDataSource(pd.DataFrame(grouped))
#print(source.data)

image_names = list( dict.fromkeys(source.data['image_name'].tolist()))
#print(image_names)

runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
#print(runtimes)

ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)

#x = [ (image_name, ressource_limit, runtime) for image_name in image_names for ressource_limit in ressource_limits for runtime in runtimes ]
#print(x)



for image_name in image_names:
    filterd=grouped[grouped['image_name'] == image_name]
    
    source = ColumnDataSource(pd.DataFrame(filterd))
    #print(source.data)

    runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
    #print(runtimes)

    ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
    #print(ressource_limits)

    x = [ (ressource_limit, runtime) for ressource_limit in ressource_limits for runtime in runtimes ]
    #print(x)

    source = ColumnDataSource(data=dict(x=x, mean=source.data['mean'].tolist(), oben=source.data['konf_intervall_oben'].tolist(), unten=source.data['konf_intervall_unten'].tolist()))

    p = figure(x_range=FactorRange(*x), width=1000)

    # use the palette to colormap based on the the x[1:2] values
    fill_color=factor_cmap('x', palette=bokeh.palettes.d3['Category10'][5], factors=runtime, start=1, end=2)

    p.vbar(x='x', top='mean', source=source, width=0.70, color=fill_color)

    p.add_layout(
      Whisker(source=source, base='x', upper="oben", lower="unten", level="overlay")
    )


    p.title.text ='Startzeit für Image ' + image_name
    p.xaxis.axis_label = 'Ressourcelimit & Runtime'
    p.yaxis.axis_label = 'Startzeit in Sekunden'
    
    p.xaxis.axis_label_text_font_size = '11pt'
    p.yaxis.axis_label_text_font_size = '11pt'
     

    output_notebook()

    show(p)

    
    

NameError: name 'runtime' is not defined

In [132]:
#####
# Vorbereitung rm time
###
grouped_std = remove_df.groupby(['image_name', 'runtime', 'ressource_limit'])['real_time'].std().reset_index(name='standartabweichung')
grouped_mean = remove_df.groupby(['image_name', 'runtime', 'ressource_limit'])['real_time'].mean().reset_index(name='mean')
#print(grouped_std)
#print(grouped_mean)

grouped = pd.merge(grouped_mean, grouped_std, on=['image_name', 'runtime', 'ressource_limit'])
#print(grouped)

# Quantiel für 35 - 1 = 34 -> 2.7284
# Quelle https://www.staff.uni-mainz.de/klenke/vorlesungen/vorl_ws19-2/tabellenwerk.pdf S7

grouped['konf_intervall_unten'] = grouped['mean'] - grouped['standartabweichung']  / math.sqrt(35) * 2.7284
grouped['konf_intervall_oben'] = grouped['mean'] + grouped['standartabweichung']  / math.sqrt(35) *  2.7284

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(grouped)

         image_name  runtime ressource_limit      mean  standartabweichung  \
0            alpine  default         default  0.493714            0.042216   
1            alpine  default             max  0.493057            0.044798   
2            alpine  default             min  0.451543            0.043925   
3            alpine     kata         default  0.691914            0.084008   
4            alpine     kata             max  0.727086            0.074514   
5            alpine     kata             min  0.639486            0.044386   
6            alpine   katafc         default  0.635257            0.171241   
7            alpine   katafc             max  0.749257            0.140480   
8            alpine   katafc             min  0.803514            0.042838   
9            alpine    runsc         default  0.490429            0.046911   
10           alpine    runsc             max  0.509257            0.065692   
11           alpine    runsc             min  0.442629          

In [133]:
#latex print
printtbl = grouped[['runtime', 'image_name', 'ressource_limit', 'mean', 'standartabweichung', 'konf_intervall_unten', 'konf_intervall_oben', ]]
print(printtbl.to_latex())

\begin{tabular}{llllrrrr}
\toprule
{} &  runtime &      image\_name & ressource\_limit &      mean &  standartabweichung &  konf\_intervall\_unten &  konf\_intervall\_oben \\
\midrule
0   &  default &          alpine &         default &  0.493714 &            0.042216 &              0.474245 &             0.513183 \\
1   &  default &          alpine &             max &  0.493057 &            0.044798 &              0.472397 &             0.513717 \\
2   &  default &          alpine &             min &  0.451543 &            0.043925 &              0.431285 &             0.471800 \\
3   &     kata &          alpine &         default &  0.691914 &            0.084008 &              0.653171 &             0.730657 \\
4   &     kata &          alpine &             max &  0.727086 &            0.074514 &              0.692721 &             0.761450 \\
5   &     kata &          alpine &             min &  0.639486 &            0.044386 &              0.619016 &             0.659956 \\
6   & 

In [140]:
########
# Sum Remove time
#######

group_sum=grouped.groupby(['ressource_limit', 'runtime'])['mean'].sum().reset_index(name='sum_mean')

#latex print
printtbl = group_sum[['ressource_limit', 'runtime', 'sum_mean']]
print(printtbl.to_latex())


source = ColumnDataSource(pd.DataFrame(group_sum))
#print(source.data)

runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
#print(runtimes)

ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)

x = [ (ressource_limit, runtime) for ressource_limit in ressource_limits for runtime in runtimes ]
print(x)

source = ColumnDataSource(data=dict(x=x, mean=source.data['sum_mean'].tolist()))
print(source.data)
p = figure(x_range=FactorRange(*x), width=1000)

# use the palette to colormap based on the the x[1:2] values
fill_color=factor_cmap('x', palette=bokeh.palettes.d3['Category10'][5], factors=runtime, start=1, end=2)

p.vbar(x='x', top='mean', source=source, width=0.70, color=fill_color)



p.title.text ='Removetime in Summe'
p.xaxis.axis_label = 'Ressourcenlimit & Runtime'
p.yaxis.axis_label = 'Removetime in Sekunden'

p.xaxis.axis_label_text_font_size = '11pt'
p.yaxis.axis_label_text_font_size = '11pt'

output_notebook()

show(p)

\begin{tabular}{lllr}
\toprule
{} & ressource\_limit &  runtime &   sum\_mean \\
\midrule
0  &         default &  default &  11.321971 \\
1  &         default &     kata &  15.770314 \\
2  &         default &   katafc &  19.335914 \\
3  &         default &    runnc &   2.828429 \\
4  &         default &    runsc &  11.875657 \\
5  &             max &  default &  11.486629 \\
6  &             max &     kata &  15.801200 \\
7  &             max &   katafc &  19.904371 \\
8  &             max &    runnc &   2.627371 \\
9  &             max &    runsc &  11.912829 \\
10 &             min &  default &  11.160886 \\
11 &             min &     kata &  15.989114 \\
12 &             min &   katafc &  19.501714 \\
13 &             min &    runnc &   2.386286 \\
14 &             min &    runsc &  11.719543 \\
\bottomrule
\end{tabular}

[('default', 'default'), ('default', 'kata'), ('default', 'katafc'), ('default', 'runnc'), ('default', 'runsc'), ('max', 'default'), ('max', 'kata'), ('max', 'kata

Loading BokehJS ...

In [144]:
######
# Removetime indivduell
####

source = ColumnDataSource(pd.DataFrame(grouped))
#print(source.data)

image_names = list( dict.fromkeys(source.data['image_name'].tolist()))
#print(image_names)

runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
#print(runtimes)

ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)

#x = [ (image_name, ressource_limit, runtime) for image_name in image_names for ressource_limit in ressource_limits for runtime in runtimes ]
#print(x)



for image_name in image_names:
    filterd=grouped[grouped['image_name'] == image_name]
    
    source = ColumnDataSource(pd.DataFrame(filterd))
    #print(source.data)

    runtimes = list( dict.fromkeys(source.data['runtime'].tolist()))
    #print(runtimes)

    ressource_limits= list( dict.fromkeys(source.data['ressource_limit'].tolist()))
    #print(ressource_limits)

    x = [ (ressource_limit, runtime) for ressource_limit in ressource_limits for runtime in runtimes ]
    #print(x)

    source = ColumnDataSource(data=dict(x=x, mean=source.data['mean'].tolist(), oben=source.data['konf_intervall_oben'].tolist(), unten=source.data['konf_intervall_unten'].tolist()))

    p = figure(x_range=FactorRange(*x), width=1000)

    # use the palette to colormap based on the the x[1:2] values
    fill_color=factor_cmap('x', palette=bokeh.palettes.d3['Category10'][5], factors=runtime, start=1, end=2)

    p.vbar(x='x', top='mean', source=source, width=0.70, color=fill_color)

    p.add_layout(
      Whisker(source=source, base='x', upper="oben", lower="unten", level="overlay")
    )


    p.title.text ='Removetime für Image ' + image_name
    p.xaxis.axis_label = 'Ressourcenlimit & Runtime'
    p.yaxis.axis_label = 'Removetime in Sekunden'
    
    p.xaxis.axis_label_text_font_size = '11pt'
    p.yaxis.axis_label_text_font_size = '11pt'
     

    output_notebook()

    show(p)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [11]:
source = ColumnDataSource(pd.DataFrame(grouped))
print(source.data)

{'image_name_runtime': array([('alpine', 'default'), ('alpine', 'runsc'), ('busybox', 'default'),
       ('busybox', 'runsc'), ('go-httpd', 'default'),
       ('go-httpd', 'runnc'), ('go-httpd', 'runsc'), ('httpd', 'default'),
       ('httpd', 'runsc'), ('mongo', 'default'), ('mongo', 'runsc'),
       ('nginx', 'default'), ('nginx', 'runsc'),
       ('node-express', 'default'), ('node-express', 'runnc'),
       ('node-express', 'runsc'), ('postgres', 'default'),
       ('postgres', 'runsc'), ('python-tornado', 'default'),
       ('python-tornado', 'runnc'), ('python-tornado', 'runsc'),
       ('redis-test', 'default'), ('redis-test', 'runnc'),
       ('redis-test', 'runsc'), ('tomcat', 'default'),
       ('tomcat', 'runsc'), ('traefik', 'default'), ('traefik', 'runsc'),
       ('ubuntu', 'default'), ('ubuntu', 'runsc')], dtype=object), 'real_time': array([1.232     , 1.12466667, 0.794     , 1.20766667, 1.787     ,
       0.916     , 2.14333333, 1.233     , 0.931     , 1.271     ,
     

In [13]:
runtimes = source.data['image_name_runtime'].tolist()

print(runtimes)

p = figure(x_range=FactorRange(*runtimes), width=2500)

colors = bokeh.palettes.d3['Category10'][5]
colors = colors + colors + colors
print(colors)

colors_itter = itertools.cycle(colors)

color_map = factor_cmap(field_name='image_name_runtime',
                    palette=colors, factors=runtimes)

p.vbar(x='image_name_runtime', top='real_time', source=source, width=0.70, color=color_map)

p.title.text ='Boottime'
p.xaxis.axis_label = 'Runtime, Image'
p.yaxis.axis_label = 'Boottime in Sekunden'

output_file("bootime_gouped_by_runtime_image.html")
output_notebook()

show(p)

[('alpine', 'default'), ('alpine', 'runsc'), ('busybox', 'default'), ('busybox', 'runsc'), ('go-httpd', 'default'), ('go-httpd', 'runnc'), ('go-httpd', 'runsc'), ('httpd', 'default'), ('httpd', 'runsc'), ('mongo', 'default'), ('mongo', 'runsc'), ('nginx', 'default'), ('nginx', 'runsc'), ('node-express', 'default'), ('node-express', 'runnc'), ('node-express', 'runsc'), ('postgres', 'default'), ('postgres', 'runsc'), ('python-tornado', 'default'), ('python-tornado', 'runnc'), ('python-tornado', 'runsc'), ('redis-test', 'default'), ('redis-test', 'runnc'), ('redis-test', 'runsc'), ('tomcat', 'default'), ('tomcat', 'runsc'), ('traefik', 'default'), ('traefik', 'runsc'), ('ubuntu', 'default'), ('ubuntu', 'runsc')]
['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']


/usr/local/lib/python3.7/dist-packages/bokeh/models/mappers.py:140: UserWarning: Palette length does not match number of factors. [('node-express', 'runsc'), ('postgres', 'default'), ('postgres', 'runsc'), ('python-tornado', 'default'), ('python-tornado', 'runnc'), ('python-tornado', 'runsc'), ('redis-test', 'default'), ('redis-test', 'runnc'), ('redis-test', 'runsc'), ('tomcat', 'default'), ('tomcat', 'runsc'), ('traefik', 'default'), ('traefik', 'runsc'), ('ubuntu', 'default'), ('ubuntu', 'runsc')] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))


Loading BokehJS ...

In [15]:
grouped = df.groupby(['runtime'])['real_time'].mean()
print(grouped)

runtime
default    1.230000
runnc      1.245333
runsc      1.311846
Name: real_time, dtype: float64


In [16]:
source = ColumnDataSource(pd.DataFrame(grouped))
print(source.data)

{'runtime': array(['default', 'runnc', 'runsc'], dtype=object), 'real_time': array([1.23      , 1.24533333, 1.31184615])}


In [18]:
runtimes = source.data['runtime'].tolist()

print(runtimes)

p = figure(x_range=FactorRange(*runtimes), width=1500)

colors = bokeh.palettes.d3['Category10'][5]
colors = colors + colors + colors

color_map = factor_cmap(field_name='runtime',
                    palette=colors, factors=runtimes)

p.vbar(x='runtime', top='real_time', source=source, width=0.70, color=color_map)

p.title.text ='Boottime'
p.xaxis.axis_label = 'Runtime, Image'
p.yaxis.axis_label = 'Boottime in Sekunden'

output_file("bootime_gouped_by_runtime_image.html")
output_notebook()

show(p)

['default', 'runnc', 'runsc']
['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']


Loading BokehJS ...